This problem is very similar to __Minimum Swaps__, wherein we were tasked with finding the minimum number of swapped items that could sort a given unsorted (1-indexed) array.  There are a number of added wrinkles to the problem, however.  

1.  The elements in the array correspond to people standing in a queue.  
2.  Any person in queue can bribe the person directly in front of them to swap positions. 
3.  Each person can bribe at most 2 others.

We could probably use our `minimumSwaps` function as a starting point, but I think I'll try to solve this in OOP style.

In [43]:
class Queue():
    """A line of people waiting for something."""
    def __init__(self, people):
        self.people = list(people)

    def swap(self, i, j):
        """Swaps Person i with Person j"""
        if self.people[j].can_bribe():
            self.people[j].bribes_made += 1
            self.people[i], self.people[j] = self.people[j], self.people[i]
            
        
    def __repr__(self):
        return str(self.people)
    
    __str__ = __repr__
    
    
class Person():
    """A person standing in this peculiar queue."""
    def __init__(self, number):
        self.number = number     # self.number never changes
        self.pos = number        # self.pos may change
        
        self.bribes_made = 0     
        
    def can_bribe(self):
        return self.bribes_made < 2
    
    def __repr__(self):
        return 'Person({})'.format(self.number)
    
    __str__ = __repr__

In [44]:
people = [Person(i) for i in range(1,7)]
people

[Person(1), Person(2), Person(3), Person(4), Person(5), Person(6)]

In [45]:
for person in people:
    print(person.bribes_made)

0
0
0
0
0
0


In [46]:
Q = Queue(people)
Q

[Person(1), Person(2), Person(3), Person(4), Person(5), Person(6)]

In [47]:
Q.swap(3, 4)
Q

[Person(1), Person(2), Person(3), Person(5), Person(4), Person(6)]

In [48]:
for person in Q.people:
    print("{} has made {} bribes".format(person, person.bribes_made))

Person(1) has made 0 bribes
Person(2) has made 0 bribes
Person(3) has made 0 bribes
Person(5) has made 1 bribes
Person(4) has made 0 bribes
Person(6) has made 0 bribes


In [49]:
Q.swap(2,3)

In [50]:
for person in Q.people:
    print("{} has made {} bribes".format(person, person.bribes_made))

Person(1) has made 0 bribes
Person(2) has made 0 bribes
Person(5) has made 2 bribes
Person(3) has made 0 bribes
Person(4) has made 0 bribes
Person(6) has made 0 bribes


In [51]:
Q.swap(1,2)

In [52]:
# As desired, once a person has made 2 bribes, no more can be made.

for person in Q.people:
    print("{} has made {} bribes".format(person, person.bribes_made))

Person(1) has made 0 bribes
Person(2) has made 0 bribes
Person(5) has made 2 bribes
Person(3) has made 0 bribes
Person(4) has made 0 bribes
Person(6) has made 0 bribes


This is a good start.  Our queue is 0-indexed, but we can probably work around that.  But how to find the minimum number of swaps required?

Well, a solution comes to mind that again makes no use of my OOP scaffolding!  We basically just need to use our old `minimumSwaps` function, with the proviso that should we find that any elements are more than 2 places in front of their initial position, we return 'Too chaotic'.  We can accomplish this in $O(n)$ time at the outset by simply iterating through the given array looking for elements that are too far removed.

In [53]:
def swap(i, j, arr):
    arr[i], arr[j] = arr[j], arr[i]
    return arr

def minimumBribes(arr):
    """Returns minimum number of swaps required to sort arr."""
    
    # our bribe-cap sentry
    for a in arr:
        if a - (arr.index(a)+1) > 2:
            return 'Too chaotic'
    
    swaps = 0
    places = {a:i for i, a in enumerate(arr)} 
    
    for i, a in enumerate(arr):
        
        if arr[i] == i+1:
            pass
            
        else:
            j = places[i+1]
            swap(i, j, arr)
            places[arr[j]] = j
            swaps += 1
        
    return swaps

In [54]:
test1 = [2,1,5,3,4]
test2 = [2,5,1,3,4]

In [55]:
print(minimumSwaps(test1))
print(minimumSwaps(test2))

3
Too chaotic


After trying this out in the grader, it seems that not all is well.  Two test cases passing, but this one is failing:

In [56]:
arr1 = [5, 1, 2, 3, 7, 8, 6, 4]
arr2 = [1, 2, 5, 3, 7, 8, 6, 4]

In [57]:
print(minimumSwaps(arr1))
print(minimumSwaps(arr2))

Too chaotic
5


The answer for `arr1` is correct, but the other apparently you be 7, rather than 5.  After thinking a bit more I made the mistake of reading in the discussion forums this brilliant hack.  Pretty much everything I've done so far is unnecessary.  All we must do is

```
total_bribes = 0   # 1 bribe <=> 1 swap
for a in array
    if a - (array.index(a) + 1) > 2
        return 'Too chaotic'
    total_bribes += (array.index(a) + 1) - a
return total_bribes
```
     

But how to count the number of times an person has been bribed?  My initial thought was to simply take the difference between the person's number and his/her final position:

In [58]:
arr2 = [1, 2, 5, 3, 7, 8, 6, 4]

bribes = 0
for a in arr2:
    dist = a - (arr2.index(a) + 1)
    if dist < 0:
        bribes += abs(dist) 
    
print(bribes)


6


But this is no good.  Let's look closer.

In [59]:
print("a".center(10), "pos".center(10), "dist".center(10))
print("====================================")
for a in arr2:
    dist = a - (arr2.index(a) + 1)
    print(
        "{}".format(a).center(10),  
        "{}".format(arr2.index(a) + 1).center(10),
        "{}".format(dist).center(10)
    )

    a         pos        dist   
    1          1          0     
    2          2          0     
    5          3          2     
    3          4          -1    
    7          5          2     
    8          6          2     
    6          7          -1    
    4          8          -4    


We can reconstruct the required swaps:

1.  5 bribes 4:  `[1, 2, 3, 5, 4, 6, 7, 8]`
2.  5 bribes 3:   `[1, 2, 5, 3, 4, 6, 7, 8]`
3.  7 bribes 6:   `[1, 2, 5, 3, 4, 7, 6, 8]`
4.  7 bribes 4:   `[1, 2, 5, 3, 7, 4, 6, 8]`
5.  8 bribes 6:   `[1, 2, 5, 3, 7, 4, 8, 6]`
6.  8 bribes 4:   `[1, 2, 5, 3, 7, 8, 4, 6]`
7.  6 bribes 4:   `[1, 2, 5, 3, 7, 8, 6, 4]`

Referring to our `dist` column from above, we find that we missed counting one of the times that 6 was bribed.  The reason why:  6 was bribed twice, but also performed one bribe.  We were only computing the net gain, not total times bribed.  How can we count the latter?

Note that every time person P gets bribed, they switch places with person P-1.  That is to say, the number of persons Q where Q > P that are ahead of P in line are incremented by 1.  Note further that P-1 will never move behind P, we are looking for a _minimal_ number of moves, so nothing superfluous allowed.  Also, note that any person having bribed P could not get any further than P-1, since no one can move more than 2 places forward.  Taken together, this gives us the following method for counting the number of times person P has been bribed:  we simply count the number of persons Q with Q>P in the range `arr[P-1 : arr.index(P)]`.  That is, the number of persons greater than P that are between the place to the left of where P began and the place where P wound up.

In [73]:
arr1 = [5, 1, 2, 3, 7, 8, 6, 4]
arr2 = [1, 2, 5, 3, 7, 8, 6, 4]
arr3 = [2, 1, 5, 3, 4]
arr4 = [2, 5, 1, 3, 4]

In [93]:
def test(func, args, expected):
    """Requires len(args) == len(expected)"""
    print("TESTING {}".format(func.__name__).center(30))
    print("="*30)
    num_failed, which_failed = 0, []
    for i, (arg, exp) in enumerate(zip(args, expected)):
        computed = func(arg)
        print("Test {}:  {}({})".format(i, func.__name__, arg))
        print("  Computed: {}".format(computed))
        print("  Expected: {}".format(exp))
        if func(arg) != exp:
            print(computed, exp)
            print("TEST {} FAILED".format(i).center(30))
            which_failed.append(i)
            num_failed += 1
        print("-"*30)
    print("="*30)
    print("The following {} {} failing:".format(num_failed, {0: 'test is', 1: 'tests are'}[num_failed > 1]))
    print("{}".format(which_failed).center(30))
        
        
test(minimumBribes, [arr1, arr2, arr3, arr4], ['Too chaotic', 7, 3, 'Too chaotic'])

    TESTING minimumBribes     
Test 0:  minimumBribes([0, 0, 0, 5, 1, 2, 3, 7, 8, 6, 4])
  Computed: 1
  Expected: Too chaotic
1 Too chaotic
        TEST 0 FAILED         
------------------------------
Test 1:  minimumBribes([0, 0, 0, 0, 1, 2, 5, 3, 7, 8, 6, 4])
  Computed: 0
  Expected: 7
0 7
        TEST 1 FAILED         
------------------------------
Test 2:  minimumBribes([0, 0, 0, 0, 2, 1, 5, 3, 4])
  Computed: 0
  Expected: 3
0 3
        TEST 2 FAILED         
------------------------------
Test 3:  minimumBribes([0, 0, 0, 0, 2, 5, 1, 3, 4])
  Computed: 0
  Expected: Too chaotic
0 Too chaotic
        TEST 3 FAILED         
------------------------------
The following 4 tests are failing:
         [0, 1, 2, 3]         


In [72]:
# second attempt
# sole change:  left index changed to max(0, a-2), since for a=1, a-2=-1
#               which wraps around to the end of the array
# Passing more tests now, but failing several due to timeout.

def minimumBribes(arr):
    total_bribes = 0
    for a in arr:
        if a - (arr.index(a) + 1) > 2:
            return 'Too chaotic'
        
        bribes = sum([b > a for b in arr[max(0, a-2):arr.index(a)+1]])

        total_bribes += bribes
        
    return total_bribes

In [68]:
def minimumBribes(arr):
    total_bribes = 0
    for a in arr:
        
        if a - (arr.index(a) + 1) > 2:
            return 'Too chaotic'
        
        for b in arr[max(0, a-2):arr.index(a)+1]:
            total_bribes += (b > a)
            
        #bribes = sum([b > a for b in arr[max(0, a-2):arr.index(a)+1]])
        #total_bribes += bribes
        
    return total_bribes

In [75]:
test(minimumBribes, [arr1, arr2, arr3, arr4], ['Too chaotic', 7, 3, 'Too chaotic'])

    TESTING minimumBribes     
Test 0:  minimumBribes([5, 1, 2, 3, 7, 8, 6, 4])
  Computed: Too chaotic
  Expected: Too chaotic
------------------------------
Test 1:  minimumBribes([1, 2, 5, 3, 7, 8, 6, 4])
  Computed: 7
  Expected: 7
------------------------------
Test 2:  minimumBribes([2, 1, 5, 3, 4])
  Computed: 3
  Expected: 3
------------------------------
Test 3:  minimumBribes([2, 5, 1, 3, 4])
  Computed: Too chaotic
  Expected: Too chaotic
------------------------------
The following 0 test is failing:
              []              


In [126]:
# rewriting

arr1 = [5, 1, 2, 3, 7, 8, 6, 4]
arr2 = [1, 2, 5, 3, 7, 8, 6, 4]
arr3 = [2, 1, 5, 3, 4]
arr4 = [2, 5, 1, 3, 4]

def minimumBribes(arr, toPrint=False):
    arr = [0] + list(arr)
    pending = set([1])
    total_bribes = 0
    for i in range(len(arr) - 1):
        if arr[i] > i+2:
            return 'Too chaotic'
        print("i={}".format(i), "arr[i]={}".format(arr[i]))
        print("pending:", pending)
        pending.discard(i)
        if i+2 < len(arr):
            pending.add(arr[i+2])
        if toPrint:
            print("pending after changes:", pending)
        for p in pending:
            print("  Checking each p in pending")
            if toPrint:
                if arr[i] > p:
                    print("    arr[i]={} > {}=p".format(arr[i], p))
                    print("    => p={} was bribed".format(p))
                else:
                    print("    No bribes")
            total_bribes += arr[i] > p
            print("total_bribes", total_bribes)
            
    return total_bribes

minimumBribes(arr2, toPrint=True)

i=0 arr[i]=0
pending: {1}
pending after changes: {1, 2}
  Checking each p in pending
    No bribes
total_bribes 0
  Checking each p in pending
    No bribes
total_bribes 0
i=1 arr[i]=1
pending: {1, 2}
pending after changes: {2, 5}
  Checking each p in pending
    No bribes
total_bribes 0
  Checking each p in pending
    No bribes
total_bribes 0
i=2 arr[i]=2
pending: {2, 5}
pending after changes: {3, 5}
  Checking each p in pending
    No bribes
total_bribes 0
  Checking each p in pending
    No bribes
total_bribes 0
i=3 arr[i]=5
pending: {3, 5}
pending after changes: {5, 7}
  Checking each p in pending
    No bribes
total_bribes 0
  Checking each p in pending
    No bribes
total_bribes 0
i=4 arr[i]=3
pending: {5, 7}
pending after changes: {5, 7, 8}
  Checking each p in pending
    No bribes
total_bribes 0
  Checking each p in pending
    No bribes
total_bribes 0
  Checking each p in pending
    No bribes
total_bribes 0
i=5 arr[i]=7
pending: {5, 7, 8}
pending after changes: {6, 7, 8}
  

4

In [99]:
test(minimumBribes, [arr1, arr2, arr3, arr4], ['Too chaotic', 7, 3, 'Too chaotic'])

    TESTING minimumBribes     
Test 0:  minimumBribes([5, 1, 2, 3, 7, 8, 6, 4])
  Computed: Too chaotic
  Expected: Too chaotic
------------------------------
Test 1:  minimumBribes([1, 2, 5, 3, 7, 8, 6, 4])
  Computed: 3
  Expected: 7
3 7
        TEST 1 FAILED         
------------------------------
Test 2:  minimumBribes([2, 1, 5, 3, 4])
  Computed: 1
  Expected: 3
1 3
        TEST 2 FAILED         
------------------------------
Test 3:  minimumBribes([2, 5, 1, 3, 4])
  Computed: Too chaotic
  Expected: Too chaotic
------------------------------
The following 2 tests are failing:
            [1, 2]            
